In [36]:
import pandas as pd

In [37]:
index_kurento_names = [
    "loadtest-webrtc-final-kurento-2p-t3medium-retry",
    "loadtest-webrtc-final-kurento-8p-t3medium-retry",
    "loadtest-webrtc-final-kurento-3p-10s-t3medium-retry",
    "loadtest-webrtc-final-kurento-3p-40s-t3medium-retry",
]

index_mediasoup_names = [
    "loadtest-webrtc-final-mediasoup-2p-t3medium-retry-2",
    "loadtest-webrtc-final-mediasoup-8p-t3medium-retry-2",
    # "loadtest-webrtc-final-mediasoup-3p-10s-t3medium-retry-2",
    # "loadtest-webrtc-final-mediasoup-3p-40s-t3medium-retry-2",
]

index_livekit_names = [
    "loadtest-webrtc-final-livekit-2p-t3medium-retry",
    "loadtest-webrtc-final-livekit-8p-t3medium-retry",
    "loadtest-webrtc-final-livekit-3p-10s-t3medium-retry",
    "loadtest-webrtc-final-livekit-3p-40s-t3medium-retry",
]

index_list_names = index_kurento_names + index_mediasoup_names + index_livekit_names

In [40]:
import functools
for index in index_list_names:
    df_users = pd.read_csv(f"dfs_final/{index}-user-join.csv")
    df_users["@timestamp"] = pd.to_datetime(df_users["@timestamp"], format='ISO8601')
    df_users = df_users.sort_values(by="@timestamp")
    # add empty int streams column
    df_users["streams_in"] = 0
    df_users["streams_out"] = 0
    df_users["publishers"] = 0
    df_users["subscribers"] = 0
    users_in_session = {}
    prev_streams_in = 0
    prev_streams_out = 0
    for i, row in df_users.iterrows():
        if row["session"] not in users_in_session:
            users_in_session[row["session"]] = {
                "publishers": 0,
                "subscribers": 0,
            }
        is_publisher = ("-2p-" in index) or ("-8p-" in index) or (int(row["participant"].replace('User', '')) <= 3)
        if is_publisher:
            users_in_session[row["session"]]["publishers"] += 1
            publishers = users_in_session[row["session"]]["publishers"]
            subscribers = users_in_session[row["session"]]["subscribers"]
            streams_in = 2 + prev_streams_in
            streams_out = 2 * publishers * (publishers - 1) + 2 * subscribers + prev_streams_out
        else:
            users_in_session[row["session"]]["subscribers"] += 1
            publishers = users_in_session[row["session"]]["publishers"]
            subscribers = users_in_session[row["session"]]["subscribers"]
            streams_in = prev_streams_in
            streams_out = 2 * publishers + prev_streams_out
        df_users.at[i, "streams_in"] = streams_in
        df_users.at[i, "streams_out"] = streams_out
        df_users.at[i, "publishers"] = functools.reduce(lambda a, b: a + b, [users_in_session[session]["publishers"] for session in users_in_session])
        df_users.at[i, "subscribers"] = functools.reduce(lambda a, b: a + b, [users_in_session[session]["subscribers"] for session in users_in_session])
        prev_streams_in = streams_in
        prev_streams_out = streams_out
    df_users.to_csv(f"dfs_final/{index}-user-join.csv", index=False)

In [26]:
for index in index_list_names:
    df = pd.read_csv(f"dfs_final/{index}.csv")
    df_users = pd.read_csv(f"dfs_final/{index}-user-join.csv")
    df_users["@timestamp"] = pd.to_datetime(df_users["@timestamp"], format='ISO8601')
    df_users = df_users.sort_values(by="@timestamp")
    df = df.sort_values(by="@timestamp")
    for ts in df['@timestamp']:
        users_at_time = df_users[df_users['@timestamp'] <= ts]
        streams = 0
        # get all different sessions in users_at_time
        sessions = users_at_time['session'].unique()
        for session in sessions:
            users = users_at_time[users_at_time['session'] == session]
            if ("-2p-" in index) or ("-8p-" in index):
                n_users_in_session = len(users)
                streams += 2 * n_users_in_session * (n_users_in_session - 1)
            else:
                # participants have a name like User1, User2, etc
                # remove the User part and keep the number
                participant_numbers = users['participant'].str.replace('User', '').astype(int)
                publishers = len(participant_numbers[participant_numbers <= 3])
                subscribers = len(participant_numbers[participant_numbers > 3])
                streams += 2 * publishers * (publishers - 1) + subscribers * publishers
        df.loc[df['@timestamp'] == ts, 'streams'] = streams
    # df["streams"] should be integer
    df["streams"] = df["streams"].astype(int)
    df.to_csv(f"dfs_final/{index}.csv", index=False)

    
    